# "Should you use synthetic data for label balancing"

After having worked with tabular synthetic data for the past 6 months, I have encountered many articles claiming that synthetic data is the solution for many (machine learning) problems. While synthetic data can serve as a useful Privacy Enhancing Technology (PET) and has shown to be useful in certain tasks, its usefulness and relevance is not always clearly assessed. An example of this is, and also the inspiration for me writing this article, is the article provided by Synthetic Data Vault (SDV) titled: "Can you use synthetic data for label balancing?" (https://sdv.dev/blog/synthetic-label-balancing/).

The article addresses a well-known issue in classification problems: imbalanced target labels. The article correctly identifies techniques like Random Oversampling (ROS) and noise injection while acknowledging their downsides (being overfitting and noisy data). However, the article then presents synthetic data as a "compelling solution" without providing enough evidence of this. While I am a fan of SDV, their generators, preprocessors, and constraints, this article overlooks important aspects validating the validity of synthetic data for these problems. Although you definitely can use synthetic data for label balancing (to answer the question of the article), the key question is whether you **should** use synthetic data and how it compares to state-of-the-art (SOTA) techniques.

Throughout this article, I aim to provide an answer to this question by comparing synthetic data produced by SDV generators against alternatives and build on top of the aformentioned article. Specifically, I compare data-level approaches such as noise injection, ROS, Synthetic Minority Over-sampling TEchnique (SMOTE), and CTGAN against the algorithm-level approach of Cost-Sensitive learning. This idea is not novel and adjacent research is available in literature. Adiputra and Wanchai (2024) for instance compare similar approaches resampling and synthetic data approaches. However, their calidation approach uses cross validation while resampling before cross validation, which is a common pitfall leading to data leakage.

This article aims to improve on this by providing a more methodologically sound approach whilst providing the intuition and explanation for practictioners that are less familiar with imbalanced classification problems.

### Notes

- Check the helper_functions
    - move _get_num_samples to init?
    - Fix scoring function for results
    - Should I use fit/transform as opposed to fit_resample/resample in the custom samplers


## Imports

Notably, to ensure a proper cross validation procedure, I use a pipeline. Specifically, the pipeline from imb_learn is used over sklearn's

In [24]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt     
import seaborn as sns

import lightgbm as lgb

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier
from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold, cross_validate

from sdv.metadata import Metadata
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer

from helper_functions import NoiseSampler, ColumnScaler, SDVSampler, get_scores

In [25]:
import warnings
warnings.filterwarnings("ignore")

## Data exploration

For this analysis, the creditcard dataset will be used from Kaggle (https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud), containing transactions and whether they were fraudulent or not. The goal is to predict whether a transaction is fraudulent or not, making it a classification task. Naturally, the amount of non-fraudulent transactions outweigh the number of fraudulent transactions resulting in an imbalanced classification task.

For this analysis, we will only be using a subset of the columns, which in this case contains mostly the Principal Components of the original data for confidentiality reasons. In addition, we also have a variable called "Time" which is the seconds from the first transaction in the dataset, 'Amount' which is the amount spent on the transaction, and 'Class' indicating whether the transactions is fraudulent or not, which also is our target variable. Ultimately, the data consists mostly of floats with the exception of our target which is binary.

In [26]:
# Select first then and last 2 columns
creditcard = pd.read_csv('../data/creditcard.csv')

# To reduce the dimensiolality of the dataset, we will only use the first 12 principal components,
# Time, Amount, and target
print(creditcard.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Next, we see how our target labels are distributed.

In [27]:
# Target label distribution rounded to 2 decimal places
round(creditcard['Class'].value_counts(normalize = True) * 100, 2)

Class
0    99.83
1     0.17
Name: proportion, dtype: float64

It becomes evident that the dataset is highly imbalanced, with  only 0.17% of transactions being fraudulent. This can be problematic for machine learning algorithms if you were to not account for this as models tend to bias towards the non-fraudulent cases given their overrepresentation in the data. As a result, a model predicting every transaction to be non-fraudulent would already result in a 99% accuracy even though every fraudulent transaction has been wrongfully predicted. This highlights the importance of carefully selecting evaluation metrics, as accuracy alone can be misleading in imbalanced classification problems.

### Data splitting

Next step is to split the data. We stratify on the target variable to ensure an even split across train/set sets.

In [28]:
X = creditcard.drop('Class', axis = 1)
y = creditcard['Class']

## Metrics

As mentioned previously, the choice of metrics is far from trivial. However, given that it is beyond the scope of this blog, I will choose an F1-score without diving too deep into the costs of misclassification.

### Algorithm

For the algorithm, I will use a LGBM Classifier. Choosing the most optimal estimator is beyond the scope of this blog. LGBM is chosen for its efficiency and relative predictive power, therefore being used consistently.

## Settings

We define the folds and parameters to optimize over as these will be consistent across resampling techniques.

In [29]:
RANDOM_STATE = 2

CV_FOLDS = StratifiedKFold(n_splits = 5, random_state = RANDOM_STATE, shuffle = True)

scorings = ['f1', 'roc_auc', 'precision', 'recall']

## Baseline performance

As a baseline for this problem to compare predictions to, we will use the DummyClassifier from sklearn. We use 'stratified' becasue choosing 'most_frequent' is probably also a natural choice, but given that we mostly evaluate on F1, all scores would be zero, making the improvement over the baseline seem better more impressive.

In [ ]:
pipeline_baseline = Pipeline(
    [('dummy', DummyClassifier(strategy = 'stratified', random_state=RANDOM_STATE)),]
)

cv_score_base = cross_validate(pipeline_baseline, X, y, cv = CV_FOLDS, scoring = scorings, 
                                n_jobs = -1)


{'fit_time': array([0.08679271, 0.08679271, 0.08679271, 0.08679271, 0.08679271]),
 'score_time': array([0.03021216, 0.03021216, 0.02011299, 0.03021216, 0.02011299]),
 'test_f1': array([0., 0., 0., 0., 0.]),
 'test_roc_auc': array([0.49912069, 0.49912069, 0.49912069, 0.49912069, 0.49912069]),
 'test_precision': array([0., 0., 0., 0., 0.]),
 'test_recall': array([0., 0., 0., 0., 0.])}

## Noise injection

The first approach mentioned in the article is noise injection. Although I have not seen it being used in practice and the article does not mention the noise generating process, a uniform sampling procedure will be used. Specifically, for each variable I will extract their minimum and maximum values and use them to sample from a uniform distribution. Therefore, the correlations between variables are overlooked and the bivariate distributions won't be correct.

Given that all variables, with exception of the target variable, are numerical this step is quite straightforward.

In [31]:
# Define the sampling pipeline
pipeline_noise = Pipeline(
    [('noise_sampler', NoiseSampler(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_noise = cross_validate(pipeline_noise, X, y, cv = CV_FOLDS, scoring = scorings, 
                                n_jobs = -1)

## ROS

In [32]:
# Define the sampling pipeline
pipeline_ros = Pipeline(
    [('ros', RandomOverSampler(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_ros = cross_validate(pipeline_ros, X, y, cv = CV_FOLDS, scoring = scorings, 
                               n_jobs = -1)

## SMOTE

In [33]:
# Define the sampling pipeline
pipeline_smote = Pipeline(
    [('scaler', ColumnScaler(['Amount', 'Time'])),
     ('smote', SMOTE(random_state = RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_smote = cross_validate(pipeline_smote, X, y, cv = CV_FOLDS, scoring = scorings, 
                                n_jobs = -1)


## SDV Generators

The SDV generators that will be compared are the CTGAN and TVAE

In [34]:
metadata = Metadata.detect_from_dataframe(X)

### CTGAN

I will use the CTGAN synthesizer in this case with default parameters and without any constraints.

In [35]:
# Define the sampling pipeline
pipeline_ctgan = Pipeline(
    [('ctgan', SDVSampler(CTGANSynthesizer, metadata, random_state= RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_ctgan = cross_validate(pipeline_ctgan, X, y, cv = CV_FOLDS, scoring = scorings, 
                               n_jobs = -1)

## TVAE Synthesizer

I will also use the TVAE Synthesizer. Again, there will be no hyperparameter tuning and no constraints.

In [36]:
# Define the sampling pipeline
pipeline_tvae = Pipeline(
    [('tvae', SDVSampler(TVAESynthesizer, metadata, random_state= RANDOM_STATE)),
     ('LGB', lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1))]
)

cv_score_tvae = cross_validate(pipeline_tvae, X, y, cv=CV_FOLDS, scoring = scorings,
                               n_jobs = -1)

## Cost sensitive learning

I will be using the inverse class frequency for this as the real costs associated with misclassifying creditcard fraud is not known in this example. These weights will be specified using the classifier's 'class_weight' parameter. Specifically, this is set to 'balanced' to achieve the inversely proportional weights. 

Given that the folds are stratified, we assume the that the assigned weights for the entire train set is (roughly) equal to the weights for each fold. This will result in the following weights being assigned:


In [37]:
X.shape[0] / (2 * np.bincount(y)) 

array([  0.50086524, 289.43800813])

The pipeline then becomes

In [38]:
# Define the sampling pipeline
pipeline_csl = Pipeline(
    [('LGB', lgb.LGBMClassifier(class_weight= 'balanced', random_state = RANDOM_STATE,
                                n_jobs = -1, verbose = -1))]
)

cv_score_csl = cross_validate(pipeline_csl, X, y, cv=CV_FOLDS, scoring = scorings,
                               n_jobs = -1)

# Results

In [39]:
print(cv_score_csl['test_f1'].mean())
cv_score_csl

0.8388369280184002


{'fit_time': array([4.70633245, 5.10437608, 4.82159638, 4.70733619, 4.71133566]),
 'score_time': array([0.36897874, 0.15098023, 0.32333207, 0.38900161, 0.397228  ]),
 'test_f1': array([0.86458333, 0.83333333, 0.83248731, 0.83838384, 0.82539683]),
 'test_roc_auc': array([0.98446967, 0.97377153, 0.9660105 , 0.9700833 , 0.98033602]),
 'test_precision': array([0.89247312, 0.80952381, 0.82828283, 0.83      , 0.85714286]),
 'test_recall': array([0.83838384, 0.85858586, 0.83673469, 0.84693878, 0.79591837])}

In [40]:
scorings

['f1', 'roc_auc', 'precision', 'recall']

In [ ]:
cv_scores = [cv_score_base, cv_score_noise, cv_score_ros, cv_score_smote,
             cv_score_ctgan, cv_score_tvae, cv_score_csl]

def display_scores(cv_scores, scorings):
   res = {}
   
   for scoring in scorings:
       res[scoring] = 
       
       
    
    

SyntaxError: incomplete input (4194102145.py, line 6)

In [ ]:
# Complete the scores 

